#  PikaPikaGenerator - Model Architecture
# 
**Progetto:** Generative Synthesis of Pokémon Sprites from Textual Descriptions  
 **Corso:** Deep Learning - Politecnico di Bari  
 **Studente:** Pasquale Alessandro Denora  
 **Professore:** Vito Walter Anelli 

#  Import e Setup Iniziale
 
 Il file inizia importando le librerie essenziali per implementare il modello deep learning:
 - **torch & torch.nn**: Framework PyTorch per reti neurali
 - **transformers**: Per utilizzare modelli BERT pre-addestrati
 - **numpy**: Per operazioni numeriche e conversioni
 - **typing**: Per type hints e migliore documentazione
 - **logging**: Per tracciare informazioni durante l'esecuzione


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
import numpy as np
from typing import Dict, Tuple, Optional, List
import logging

logger = logging.getLogger(__name__)

#  Classe TextEncoder - Encoder Testuale con BERT
 
 La classe `TextEncoder` (righe 17-37) implementa l'encoder testuale basato su BERT:
 
 **Inizializzazione (righe 20-33)**:
 - Carica il modello BERT pre-addestrato specificato in config
 - Crea un layer di proiezione per adattare la dimensione output di BERT alla dimensione desiderata
 - Include LayerNorm e Dropout per stabilizzazione
 
 **Parametri config utilizzati**:
 - `config['model']['encoder']['model_name']`: nome del modello BERT (es: "prajjwal1/bert-mini")
 - `config['model']['encoder']['hidden_dim']`: dimensione delle feature testuali finali
 - `config['model']['encoder']['dropout']`: tasso di dropout per regolarizzazione


In [ ]:
class TextEncoder(nn.Module):
    """Encoder testuale basato su BERT con possibilità di fine-tuning"""
    
    def __init__(self, config: Dict):
        super().__init__()
        self.config = config['model']['encoder']
        
        # In questa fase carico il modello pre-addestrato
        self.bert = AutoModel.from_pretrained(self.config['model_name'])
        self.bert_dim = self.bert.config.hidden_size
        
        # Layer di per adattare la dimensione dell'output alla dimesione desiderata
        self.projection = nn.Sequential(
            nn.Linear(self.bert_dim, self.config['hidden_dim']),
            nn.LayerNorm(self.config['hidden_dim']),
            nn.Dropout(self.config['dropout'])
        )
        
        # Opzione Facoltativa per bloccare i pesi di BERT all'inizio per evitare aggiornamenti
        self.freeze_bert_layers(freeze=True)


#  Freeze/Unfreeze BERT e Forward Pass
 
 **Metodo freeze_bert_layers** (righe 38-41): Permette di congelare i parametri BERT per evitare aggiornamenti durante il training iniziale. Questo è utile per:
 - Stabilizzare il training nelle prime epoche
 - Ridurre l'uso di memoria
 - Evitare il "catastrophic forgetting" del modello pre-addestrato
 
 **Metodo forward** (righe 43-64): Elabora le sequenze di token attraverso BERT e proietta l'output:
 1. Passa input_ids e attention_mask a BERT
 2. Proietta la dimensione da BERT_dim a hidden_dim
 3. Restituisce sia sequence_output (per ogni token) che pooled_output (solo CLS token)


In [ ]:
def freeze_bert_layers(self, freeze: bool = True):
    """Congela o sgrava i parametri di BERT per il training"""
    for param in self.bert.parameters():
        param.requires_grad = not freeze

def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Forward pass attraverso l'encoder testuale
    Returns: (sequence_output, pooled_output)
    """
    # Eseguo il forward su BERT
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=True
    )
    
    # Prende l'output per ogni token (sequence_output))
    sequence_output = outputs.last_hidden_state  # [batch, seq_len, bert_dim]
    
    # Proietta alla dimensione desiderata
    sequence_output = self.projection(sequence_output)  # [batch, seq_len, hidden_dim]
    
    # ottiene  output (CLS token)
    pooled_output = sequence_output[:, 0, :]  # [batch, hidden_dim]
    
    return sequence_output, pooled_output

#  Classe MultiHeadAttention - Meccanismo di Attenzione
 
 La classe `MultiHeadAttention` (righe 67-124) implementa il meccanismo di attenzione multi-head per allineare features testuali e visuali:
 
 **Inizializzazione (righe 70-82)**:
 - Definisce il numero di attention heads (8 heads fissi)
 - Crea layer di proiezione per Query, Key e Value
 - Calcola head_dim dividendo hidden_dim per num_heads
 
 **Parametri utilizzati**:
 - `config['model']['encoder']['hidden_dim']`: dimensione delle feature
 - `config['model']['encoder']['dropout']`: dropout per attention weights

In [ ]:
class MultiHeadAttention(nn.Module):
    """Meccanismo di multi-head attention per allineamento tra testo e immagini"""
    
    def __init__(self, config: Dict):
        super().__init__()
        self.hidden_dim = config['model']['encoder']['hidden_dim']
        self.num_heads = 8
        self.head_dim = self.hidden_dim // self.num_heads
        
        # Layer di proiezione per Query, Key, Value 
        self.q_proj = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.k_proj = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.v_proj = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.out_proj = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.dropout = nn.Dropout(config['model']['encoder']['dropout'])


#  Forward Pass Multi-Head Attention
 
 Il metodo `forward` (righe 84-124) implementa il meccanismo di attention completo:
 
 **Step-by-step**:
 1. **Proiezione QKV** (righe 98-100): Proietta input in Query, Key, Value e ridimensiona per multi-head
 2. **Calcolo scores** (righe 103): Calcola attention scores con dot-product scalato
 3. **Applicazione mask** (righe 106-108): Maschera padding tokens con valori molto negativi
 4. **Softmax e dropout** (righe 112-113): Normalizza scores e applica dropout
 5. **Context computation** (righe 116): Calcola context come weighted sum dei values
 6. **Output projection** (righe 119-123): Ricompone multi-head e proietta output finale
 
 **Output**: Restituisce sia l'output finale che gli attention weights (per visualizzazione)


In [ ]:
def forward(
    self, 
    query: torch.Tensor, 
    key: torch.Tensor, 
    value: torch.Tensor,
    mask: Optional[torch.Tensor] = None
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    Applica multi-head attention
    Returns: (output, attention_weights)
    """
    batch_size, seq_len = key.shape[:2]
    
    # Proietta e ridimensiona per multi-head attention
    Q = self.q_proj(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
    K = self.k_proj(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
    V = self.v_proj(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
    
    # Calcola attention scores
    scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_dim)
    
    # Applica maschera se fornita
    if mask is not None:
        mask = mask.unsqueeze(1).unsqueeze(1)  # Add head dimension
        scores = scores.masked_fill(mask == 0, -1e9)
    
    # Applica softmax per ottenere i attention weights
    attention_weights = F.softmax(scores, dim=-1)
    attention_weights = self.dropout(attention_weights)
    
    # Moltiplica i weights con i valori
    context = torch.matmul(attention_weights, V)
    
    # Rifirmatta e proietta l'output
    context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.hidden_dim)
    output = self.out_proj(context)
    
    # Media degli attention weights tra gli heads, utile per la visualization
    attention_weights = attention_weights.mean(dim=1)
    
    return output, attention_weights

#  Classe ResidualBlock - Blocchi Residuali per il Generatore
 
 La classe `ResidualBlock` (righe 128-165) implementa blocchi residuali con connessioni skip per il generatore CNN:
 
 **Caratteristiche (righe 131-150)**:
 - Due convoluzionali 3x3 con BatchNorm
 - Connessione shortcut per preservare gradiente
 - Opzione di upsampling per aumentare risoluzione spaziale
 - Se in_channels ≠ out_channels, adatta shortcut con conv 1x1
 
 **Forward pass (righe 152-165)**:
 - Applica due conv + batchnorm + relu
 - Se upsample=True, raddoppia dimensioni spaziali
 - Somma output con residual (connessione skip)


In [ ]:
class ResidualBlock(nn.Module):
    """Blocco Residuo per il generatore CNN"""
    
    def __init__(self, in_channels: int, out_channels: int, upsample: bool = False):
        super().__init__()
        self.upsample = upsample
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Connessione Shortcut
        if in_channels != out_channels or upsample:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()
        
        if upsample:
            self.upsample_layer = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

def forward(self, x: torch.Tensor) -> torch.Tensor:
    residual = x
    
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    
    if self.upsample:
        out = self.upsample_layer(out)
        residual = self.upsample_layer(residual)
    
    residual = self.shortcut(residual)
    out = F.relu(out + residual)
    
    return out

#  Classe SpriteGenerator - Generatore CNN Avanzato
 
 La classe `SpriteGenerator` (righe 168-235) è il cuore della generazione di immagini:
 
 **Inizializzazione (righe 171-208)**:
 - Combina features testuali + rumore casuale
 - Parte da feature map 4x4 piccola
 - Utilizza blocchi residuali per upsampling progressivo
 - Arriva alla dimensione finale specificata in config
 
 **Parametri config utilizzati**:
 - `config['model']['encoder']['hidden_dim']`: dimensione features testuali
 - `config['model']['generator']['noise_dim']`: dimensione vettore rumore  
 - `config['model']['generator']['base_channels']`: canali base del generatore
 - `config['model']['generator']['output_size']`: dimensione finale sprite (320x320)


In [ ]:
class SpriteGenerator(nn.Module):
    """Generatore CNN avanzato con connessioni residuali"""
    
    def __init__(self, config: Dict):
        super().__init__()
        self.text_dim = config['model']['encoder']['hidden_dim']
        self.noise_dim = config['model']['generator']['noise_dim']
        self.base_channels = config['model']['generator']['base_channels']
        self.output_size = config['model']['generator']['output_size']
        
        # Dimensione Spaziale iniziale
        self.init_size = 4  # Dimensione iniziale dell'immagine (4x4)
        
        # Proiezione Iniziale
        self.fc = nn.Sequential(
            nn.Linear(self.text_dim + self.noise_dim, self.base_channels * self.init_size * self.init_size),
            nn.BatchNorm1d(self.base_channels * self.init_size * self.init_size),
            nn.ReLU(inplace=True)
        )

#  Upsampling Progressivo e Generazione Finale
 
 **Blocchi residuali con upsampling** (righe 189-196): Serie di 6 blocchi che aumentano progressivamente la risoluzione:
 - 4x4 → 8x8 → 16x16 → 32x32 → 64x64 → 128x128 → 256x256
 - Ogni blocco dimezza il numero di canali per ridurre complessità
 - Usa bilinear upsampling per smooth scaling
 
 **Convoluzioni finali** (righe 199-205):
 - Riduce a 3 canali RGB
 - Usa Tanh per output in range [-1,1] (matching normalizzazione input)
 - Adaptive pooling per garantire esatta dimensione output_size
 
 **Forward pass** (righe 210-235): Gestisce l'intera pipeline di generazione


In [ ]:
# Blocchi residuali con upsampling progressivo fino alla dimensione finale
self.blocks = nn.ModuleList([
    ResidualBlock(self.base_channels, self.base_channels, upsample=True),      # 4x4 -> 8x8
    ResidualBlock(self.base_channels, self.base_channels // 2, upsample=True), # 8x8 -> 16x16
    ResidualBlock(self.base_channels // 2, self.base_channels // 4, upsample=True), # 16x16 -> 32x32
    ResidualBlock(self.base_channels // 4, self.base_channels // 8, upsample=True), # 32x32 -> 64x64
    ResidualBlock(self.base_channels // 8, self.base_channels // 16, upsample=True), # 64x64 -> 128x128
    ResidualBlock(self.base_channels // 16, 64, upsample=True), # 128x128 -> 256x256
])

# Convoluzione finale per generare immagine RGB con valori in [-1,1]
self.final_conv = nn.Sequential(
    nn.Conv2d(64, 32, 3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(inplace=True),
    nn.Conv2d(32, 3, 3, padding=1),
    nn.Tanh()
)

# Pooling adattivo per ottenere esttamente la dimensione dell'output_size
self.adaptive_pool = nn.AdaptiveAvgPool2d((self.output_size, self.output_size))

def forward(self, text_features: torch.Tensor, noise: Optional[torch.Tensor] = None) -> torch.Tensor:
    """Generate sprite from text features and noise"""
    batch_size = text_features.shape[0]
    
    # Se il rumore non viene fornito, viene generato casualmente
    if noise is None:
        noise = torch.randn(batch_size, self.noise_dim, device=text_features.device)
    
    # Concatena features testuali e rumore
    combined = torch.cat([text_features, noise], dim=1)
    
    # Proiezione iniziale e reshape in feature map
    x = self.fc(combined)
    x = x.view(batch_size, self.base_channels, self.init_size, self.init_size)
    
    # Unshampling progressivo tramite blocchi residuali
    for block in self.blocks:
        x = block(x)
    
    # Convoluzioni finali per immagine RGB
    x = self.final_conv(x)
    
    # Pooling adattivo per ottenere la dimensione finale desiderata
    x = self.adaptive_pool(x)
    
    return x


#  Classe PikaPikaGenerator - Modello Completo
 
 La classe `PikaPikaGenerator` (righe 238-357) orchestra tutti i componenti:
 
 **Inizializzazione (righe 241-251)**:
 - Crea TextEncoder, MultiHeadAttention, SpriteGenerator
 - Carica il tokenizer per preprocessing testo
 - Integra tutti i componenti in un'unica architettura
 
 **Forward pass completo** (righe 253-283):
 1. Encode del testo attraverso BERT
 2. Applica self-attention per raffinare features
 3. Genera sprite usando il generatore CNN
 4. Restituisce immagine + attention weights per analisi

In [ ]:
class PikaPikaGenerator(nn.Module):
    """Modello completo per text-to sprite generation"""
    
    def __init__(self, config: Dict):
        super().__init__()
        self.config = config
        
        # Componenti iniziali
        self.text_encoder = TextEncoder(config)
        self.attention = MultiHeadAttention(config)
        self.generator = SpriteGenerator(config)
        
        # Carica il Tokenizer per la tokenizzazione del testo
        self.tokenizer = AutoTokenizer.from_pretrained(config['model']['encoder']['model_name'])

def forward(
    self, 
    input_ids: torch.Tensor, 
    attention_mask: torch.Tensor,
    noise: Optional[torch.Tensor] = None
) -> Dict[str, torch.Tensor]:
    """
    Forward pass attraverso il model completo
    Restituisce un dizionario con lle immagini generate e gli attention weights
    """
    # Encoder testuale
    sequence_output, pooled_output = self.text_encoder(input_ids, attention_mask)
    
    # Applica self-attention per rifinire le features testuali
    attended_features, attention_weights = self.attention(
        query=pooled_output.unsqueeze(1),
        key=sequence_output,
        value=sequence_output,
        mask=attention_mask
    )
    
    attended_features = attended_features.squeeze(1)  # Rimuove la dimensione del sequence length
    
    # Generazione sprite
    generated_image = self.generator(attended_features, noise)
    
    return {
        'generated_image': generated_image,
        'attention_weights': attention_weights,
        'text_features': attended_features
    }

#  Metodi di Utilità per Generazione e Visualizzazione
 
 **Metodo generate** (righe 285-317): Interfaccia semplificata per generare sprite da testo:
 - Tokenizza automaticamente il testo input
 - Gestisce device placement (CPU/GPU)
 - Denormalizza output da [-1,1] a [0,255] per visualizzazione
 - Restituisce numpy array pronto per essere mostrato
 
 **Metodo get_attention_visualization** (righe 319-357): Per analisi dei pattern di attenzione:
 - Restituisce immagine generata + tokens + attention weights
 - Utile per capire quali parole influenzano parti specifiche dell'immagine
 - Essenziale per debugging e interpretabilità del modello


In [ ]:
def generate(
    self, 
    text: str, 
    noise: Optional[torch.Tensor] = None,
    device: str = 'cpu'
) -> np.ndarray:
    """Genera sprite dalle descrizioni testuali"""
    self.eval()
    
    # Tokenizza il testo
    encoding = self.tokenizer(
        text,
        max_length=self.config['model']['encoder']['max_length'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Sposta i tensori sul dispositivo specificato
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = self.forward(input_ids, attention_mask, noise)
        generated_image = outputs['generated_image']
    
    # Converte l'immagine generata in un array NumPy
    image = generated_image.squeeze(0).cpu()
    image = (image + 1) / 2  # Denormalize from [-1, 1] to [0, 1]
    image = image.permute(1, 2, 0).numpy()
    image = (image * 255).astype(np.uint8)
    
    return image

def get_attention_visualization(
    self, 
    text: str,
    device: str = 'cpu'
) -> Tuple[np.ndarray, List[str], np.ndarray]:
    """Ottiene gli attention weights e i tokens per visualizzazione"""
    self.eval()
    
    # Tokenizza il testo
    encoding = self.tokenizer(
        text,
        max_length=self.config['model']['encoder']['max_length'],
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Ottiene i tokens
    tokens = self.tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])
    
    # Sposta i tensori sul dispositivo specificato
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = self.forward(input_ids, attention_mask)
        attention_weights = outputs['attention_weights']
        generated_image = outputs['generated_image']
    
    # Processa gli attention weights
    attention_weights = attention_weights.squeeze().cpu().numpy()
    
    # Converte l'immagine generata in un array NumPy
    image = generated_image.squeeze(0).cpu()
    image = (image + 1) / 2
    image = image.permute(1, 2, 0).numpy()
    image = (image * 255).astype(np.uint8)
    
    return image, tokens, attention_weights

#  Classe Discriminator - Valutazione Qualità Immagini
 
 La classe `Discriminator` (righe 360-406) valuta la qualità delle immagini generate:
 
 **Architettura (righe 367-390)**:
 - Progressive downsampling: 320x320 → 160x160 → 80x80 → 40x40 → 20x20 → 10x10
 - Ogni blocco raddoppia i canali e dimezza risoluzione spaziale
 - Global average pooling per ridurre a feature vector
 - Classificatore finale per score di realismo
 
 **Metodo _make_block** (righe 392-399): Crea blocchi convoluzionali standard:
 - Conv2d con stride 2 per downsampling
 - BatchNorm per stabilizzazione
 - LeakyReLU per non-linearità (meglio per discriminatori)

In [ ]:
class Discriminator(nn.Module):
    """Discriminator per la valutazione della qualità delle immagini generate"""
    
    def __init__(self, config: Dict):
        super().__init__()
        self.input_size = config['model']['generator']['output_size']
        
        # Progressive downsampling con blocchi convoluzionali
        self.blocks = nn.Sequential(
            # 215x215 -> 107x107
            self._make_block(3, 64, downsample=True),
            # 107x107 -> 53x53
            self._make_block(64, 128, downsample=True),
            # 53x53 -> 26x26
            self._make_block(128, 256, downsample=True),
            # 26x26 -> 13x13
            self._make_block(256, 512, downsample=True),
            # 13x13 -> 6x6
            self._make_block(512, 512, downsample=True),
        )
        
        # Media pooling per ridurre le dimensioni
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Classificatore finale
        self.classifier = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1)
        )

def _make_block(self, in_channels: int, out_channels: int, downsample: bool = True):
    """Crea un blocco convoluzionale con opzioni di downsampling"""
    layers = [
        nn.Conv2d(in_channels, out_channels, 4, 2 if downsample else 1, 1),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2, inplace=True)
    ]
    return nn.Sequential(*layers)

def forward(self, x: torch.Tensor) -> torch.Tensor:
    """Forward pass attraverso il discriminatore"""
    features = self.blocks(x)
    features = self.global_pool(features).view(features.size(0), -1)
    output = self.classifier(features)
    return output

#  Funzione create_model - Factory Pattern
 
 La funzione `create_model` (righe 409-433) è la factory function per creare e inizializzare il modello:
 
 **Inizializzazione pesi** (righe 413-423):
 - Xavier uniform per Conv2d, ConvTranspose2d, Linear
 - Zero bias per tutti i layer
 - Pesi a 1 e bias a 0 per BatchNorm
 - Applica solo al generatore (encoder BERT già pre-addestrato)
 
 **Logging informazioni** (righe 426-433):
 - Conta parametri totali e trainable
 - Utile per monitorare complessità modello
 - Aiuta a diagnosticare problemi di memoria


In [ ]:
def create_model(config: Dict) -> PikaPikaGenerator:
    """Funzione per creare e inizializzare il modello PikaPikaGenerator"""
    model = PikaPikaGenerator(config)
    
    # Inizializza i weights
    def init_weights(m):
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.Linear)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
    
    model.generator.apply(init_weights)
    
    # Logging delle informazioni sul modello
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    logger.info(f"Created PikaPikaGenerator model")
    logger.info(f"Total parameters: {total_params:,}")
    logger.info(f"Trainable parameters: {trainable_params:,}")
    
    return model


#  Script di Test dell'Architettura
 
 Il blocco finale (righe 436-484) permette di testare l'architettura quando si esegue il file direttamente:
 
 **Test completi eseguiti**:
 1. **Caricamento config**: Prova a caricare config.yaml, fallback a config di test
 2. **Creazione modello**: Testa l'istanziazione di tutti i componenti
 3. **Forward pass**: Verifica che il forward pass funzioni senza errori
 4. **Shape checking**: Controlla che le dimensioni output siano corrette
 5. **Generazione testo**: Testa l'interfaccia semplificata generate()
 
 Questo è essenziale per verificare che l'architettura sia implementata correttamente prima del training.


In [ ]:
if __name__ == "__main__":
    # Test di creazione del modello e forward pass
    import yaml
    
    # Esempio config per il testing
    test_config = {
        'model': {
            'encoder': {
                'model_name': 'prajjwal1/bert-mini',
                'hidden_dim': 256,
                'max_length': 128,
                'dropout': 0.1
            },
            'generator': {
                'noise_dim': 100,
                'base_channels': 512,
                'output_size': 215
            }
        }
    }
    
    try:
        with open('configs/config.yaml', 'r') as f:
            config = yaml.safe_load(f)
    except FileNotFoundError:
        print("Config file not found, using test config")
        config = test_config
    
    model = create_model(config)
    
    # Test forward pass
    batch_size = 2
    seq_len = 128
    
    input_ids = torch.randint(0, 1000, (batch_size, seq_len))
    attention_mask = torch.ones(batch_size, seq_len)
    
    print("Testing forward pass...")
    outputs = model(input_ids, attention_mask)
    
    print(f"Generated image shape: {outputs['generated_image'].shape}")
    print(f"Attention weights shape: {outputs['attention_weights'].shape}")
    
    # Generazione del testo
    print("Testing text generation...")
    test_text = "A small yellow electric mouse Pokemon with red cheeks"
    image = model.generate(test_text)
    print(f"Generated sprite shape: {image.shape}")
    print("Architecture test completed successfully!")

#  Riepilogo Parametri Config Utilizzati
 
 Il file `architecture.py` utilizza questi parametri dal file `config.yaml`:
 
 **Sezione `model.encoder`**:
 - `model_name`: Modello BERT pre-addestrato ("prajjwal1/bert-mini")
 - `hidden_dim`: Dimensione feature testuali (512 nel tuo config)  
 - `max_length`: Lunghezza massima sequenze (128)
 - `dropout`: Tasso dropout per regolarizzazione (0.1)
 - `attention_heads`: Numero heads per attention (12, non usato nel file mostrato)
 - `attention_layers`: Numero layer attention (6, non usato nel file mostrato)
 
# **Sezione `model.generator`**:
 - `noise_dim`: Dimensione vettore rumore casuale (256)
 - `base_channels`: Canali base del generatore (768)
 - `output_size`: Dimensione finale sprite (320x320)
 - `text_dim`: Dimensione feature testuali input (512)
 - `activation`: Funzione attivazione ("leaky_relu")
 - `normalization`: Tipo normalizzazione ("spectral")
 - `use_self_attention`: Se usare self-attention (true)
 
 **Architettura Modulare**:
 Il design permette di facilmente:
 - Modificare dimensioni attraverso config
 - Sostituire componenti (es. diverso encoder BERT)
- Aggiungere nuove funzionalità senza rompere esistente
 - Testare diverse configurazioni rapidly


#  Conclusioni - Architettura PikaPikaGenerator
 
 L'architettura implementata in `architecture.py` è un modello encoder-decoder avanzato con le seguenti caratteristiche:
 
#  **Componenti Principali**:
 1. **TextEncoder**: BERT pre-addestrato + proiezione per encoding testuale
 2. **MultiHeadAttention**: Meccanismo attenzione per allineamento testo-immagine
 3. **SpriteGenerator**: CNN generativa con blocchi residuali e upsampling progressivo
 4. **Discriminator**: Valutatore qualità immagini per training adversarial
 

#  **Pipeline di Generazione**:
 1. **Input**: Descrizione testuale Pokemon
 2. **Encoding**: BERT → proiezione → attention-refined features  
 3. **Generation**: Features + rumore → upsampling progressivo → sprite 320x320
 4. **Output**: Immagine RGB + attention weights